# Curated Atlas Query (Python)

In [11]:
import pandas as pd
pd.options.display.notebook_repr_html = False
# Convert to markdown using
# poetry run jupyter nbconvert demo.ipynb --to markdown --TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}' --output README

## Importing the package

In [12]:
from curated_atlas_query_py import get_metadata, get_anndata

## Getting the metadata
The `get_metadata()` function returns a database connection and a DuckDB table.
The table can be used to query the metadata, while the connection's main purpose is to be closed when you are finished.

In [13]:
conn, table = get_metadata()
table

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

┌────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┐
│       .cell        │     sample_id_db     │       .sample        │ … │ n_cell_type_in_tis…  │ n_tissue_in_cell_t…  │
│      varchar       │       varchar        │       varchar        │   │        int64         │        int64         │
├────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┤
│ AAACCTGAGAGACGAA_1 │ 8a0fe0928684d6765d…  │ 5f20d7daf6c42f4f91…  │ … │                 NULL │                 NULL │
│ AAACCTGAGTTGTCGT_1 │ 8a0fe0928684d6765d…  │ 5f20d7daf6c42f4f91…  │ … │                 NULL │                 NULL │
│ AAACCTGCAGTCGATT_1 │ 02eb2ebcb5f802e271…  │ 5f20d7daf6c42f4f91…  │ … │                 NULL │                 NULL │
│ AAACCTGCAGTTCATG_1 │ 02eb2ebcb5f802e271…  │ 5f20d7daf6c42f4f91…  │ … │                 NULL │                 NULL │
│ AAACCTGGTCTAAACC_1 │ 8a0fe0928684d6765d…  │ 5f

### Querying the metadata
The DuckDB table can be queried using a number of methods [described here](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyRelation). In particular:
* [`.filter()`](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyRelation.filter): filters the metadata using a string expression
* [`.aggregate()`](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyRelation.aggregate): groups by one or more columns, and calculates some aggregate statistics such as counts
* [`.fetchdf()`](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyRelation.fetchdf): Executes the query and returns it as a pandas DataFrame

In [14]:
table.aggregate("tissue, file_id, COUNT(*) as n", group_expr="tissue, file_id").fetchdf()

                    tissue                               file_id        n
0                    blood  07beec85-51be-4d73-bb80-8f85b7b643d5  1399186
1         cortex of kidney  38eaca52-0ce4-4be2-8173-c6bab05f308a   122914
2            renal medulla  38eaca52-0ce4-4be2-8173-c6bab05f308a    57086
3            renal papilla  38eaca52-0ce4-4be2-8173-c6bab05f308a    20338
4            adrenal gland  5c1cc788-2645-45fb-b1d9-2f43d368bba8    95588
..                     ...                                   ...      ...
886          renal medulla  ffc36957-efef-4c98-879c-833215e850a9    12531
887            conjunctiva  343f46f2-7cdd-4da8-bc7f-50a18b2c0e8e     2084
888       cortex of kidney  f7e94dbb-8638-4616-aaf9-16e2212c369f     4628
889  mesenteric lymph node  59dfc135-19c1-4380-a9e8-958908273756    13958
890           renal pelvis  f7e94dbb-8638-4616-aaf9-16e2212c369f      163

[891 rows x 3 columns]

In [15]:
table.filter("ethnicity == 'African'")

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┐
│        .cell         │     sample_id_db     │       .sample        │ … │ n_cell_type_in_tis…  │ n_tissue_in_cell_t…  │
│       varchar        │       varchar        │       varchar        │   │        int64         │        int64         │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┤
│ AGGGAGTAGCGTTTAC_S…  │ 9da02eab40e49d1a07…  │ 20071ec5a126508641…  │ … │                   28 │                   32 │
│ ATTGGACAGCCGATTT_F…  │ 89ec472baa9d514068…  │ 4fc10a6b85e5fa688b…  │ … │                   28 │                   32 │
│ CCCAGTTCATACCATG_S…  │ 26750b2a06c447f7f2…  │ 055e5172053464e8ef…  │ … │                   28 │                   32 │
│ TGGACGCAGTGATCGG_S…  │ 26750b2a06c447f7f2…  │ 055e5172053464e8ef…  │ … │                   28 │                   32 │
│ ACGGTTACAGTCTTCC_S…  │ c87e74c

In [16]:
query = table.filter("""
    ethnicity == 'African'
    AND assay LIKE '%10%'
    AND tissue == 'lung parenchyma'
    AND cell_type LIKE '%CD4%'
""")
query

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┐
│        .cell         │     sample_id_db     │       .sample        │ … │ n_cell_type_in_tis…  │ n_tissue_in_cell_t…  │
│       varchar        │       varchar        │       varchar        │   │        int64         │        int64         │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┤
│ ACAGCCGGTCCGTTAA_F…  │ 33cdeb84ae1462d723…  │ 4fc10a6b85e5fa688b…  │ … │                   28 │                   31 │
│ GGGAATGAGCCCAGCT_F…  │ 33cdeb84ae1462d723…  │ 4fc10a6b85e5fa688b…  │ … │                   28 │                   32 │
│ TCTTCGGAGTAGCGGT_F…  │ 33cdeb84ae1462d723…  │ 4fc10a6b85e5fa688b…  │ … │                   28 │                   32 │
│ CCTTACGAGAGCTGCA_F…  │ 33cdeb84ae1462d723…  │ 4fc10a6b85e5fa688b…  │ … │                   28 │                   32 │
│ ATCTACTCAATGGAAT_F…  │ 33cdeb8

## Extracting Counts

Once you're happy with your query, you can pass it into `get_anndata()` to obtain an AnnData object:

In [17]:
get_anndata(query, assays=["counts"])

AnnData object with n_obs × n_vars = 1571 × 60661
    obs: '.cell', 'sample_id_db', '.sample', '.sample_name', 'assay', 'assay_ontology_term_id', 'file_id_db', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'ethnicity', 'ethnicity_ontology_term_id', 'file_id', 'is_primary_data.x', 'organism', 'organism_ontology_term_id', 'sample_placeholder', 'sex', 'sex_ontology_term_id', 'tissue', 'tissue_ontology_term_id', 'tissue_harmonised', 'age_days', 'dataset_id', 'collection_id', 'cell_count', 'dataset_deployments', 'is_primary_data.y', 'is_valid', 'linked_genesets', 'mean_genes_per_cell', 'name', 'published', 'revision', 'schema_version', 'tombstone', 'x_normalization', 'created_at.x', 'published_at', 'revised_at', 'updated_at.x', 'filename', 'filetype', 's3_uri', 'user_submitted', 'created_at.y', 'updated_at.y', 'cell_type_harmonised', 'confidence_class', 'cell_annotation_azimuth_l2', 'cell_annotati

You can query counts scaled per million. This is helpful if just few genes are of interest:

In [18]:
get_anndata(query, assays=['cpm'])

AnnData object with n_obs × n_vars = 1571 × 60661
    obs: '.cell', 'sample_id_db', '.sample', '.sample_name', 'assay', 'assay_ontology_term_id', 'file_id_db', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'ethnicity', 'ethnicity_ontology_term_id', 'file_id', 'is_primary_data.x', 'organism', 'organism_ontology_term_id', 'sample_placeholder', 'sex', 'sex_ontology_term_id', 'tissue', 'tissue_ontology_term_id', 'tissue_harmonised', 'age_days', 'dataset_id', 'collection_id', 'cell_count', 'dataset_deployments', 'is_primary_data.y', 'is_valid', 'linked_genesets', 'mean_genes_per_cell', 'name', 'published', 'revision', 'schema_version', 'tombstone', 'x_normalization', 'created_at.x', 'published_at', 'revised_at', 'updated_at.x', 'filename', 'filetype', 's3_uri', 'user_submitted', 'created_at.y', 'updated_at.y', 'cell_type_harmonised', 'confidence_class', 'cell_annotation_azimuth_l2', 'cell_annotati

We can query a subset of genes. Notice how the result only has `nvars = 1`:

In [19]:
anndata = get_anndata(query, features = ['PUM1'], repository='file:///vast/projects/human_cell_atlas_py/anndata')
anndata

AnnData object with n_obs × n_vars = 1571 × 1
    obs: '.cell', 'sample_id_db', '.sample', '.sample_name', 'assay', 'assay_ontology_term_id', 'file_id_db', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'ethnicity', 'ethnicity_ontology_term_id', 'file_id', 'is_primary_data.x', 'organism', 'organism_ontology_term_id', 'sample_placeholder', 'sex', 'sex_ontology_term_id', 'tissue', 'tissue_ontology_term_id', 'tissue_harmonised', 'age_days', 'dataset_id', 'collection_id', 'cell_count', 'dataset_deployments', 'is_primary_data.y', 'is_valid', 'linked_genesets', 'mean_genes_per_cell', 'name', 'published', 'revision', 'schema_version', 'tombstone', 'x_normalization', 'created_at.x', 'published_at', 'revised_at', 'updated_at.x', 'filename', 'filetype', 's3_uri', 'user_submitted', 'created_at.y', 'updated_at.y', 'cell_type_harmonised', 'confidence_class', 'cell_annotation_azimuth_l2', 'cell_annotation_b

We can access the metadata using normal anndata conventions:

In [20]:
anndata.obs

                        .cell                      sample_id_db  \
0     ACAGCCGGTCCGTTAA_F02526  33cdeb84ae1462d723c19af1bea2a366   
1     GGGAATGAGCCCAGCT_F02526  33cdeb84ae1462d723c19af1bea2a366   
2     TCTTCGGAGTAGCGGT_F02526  33cdeb84ae1462d723c19af1bea2a366   
3     CCTTACGAGAGCTGCA_F02526  33cdeb84ae1462d723c19af1bea2a366   
4     ATCTACTCAATGGAAT_F02526  33cdeb84ae1462d723c19af1bea2a366   
...                       ...                               ...   
1566  TACTTACGTAATAGCA_F02526  33cdeb84ae1462d723c19af1bea2a366   
1567    AGATAGAGTGCCTTCT_SC84  21ef23ac07391c64cadc78e16511effa   
1568    CGCGGTATCCGCGCAA_SC24  9dfbd16390b119392af9406561cb664f   
1569    TACAACGTCAGCATTG_SC84  21ef23ac07391c64cadc78e16511effa   
1570  CATTCGCTCAATACCG_F02526  33cdeb84ae1462d723c19af1bea2a366   

                               .sample  \
0     4fc10a6b85e5fa688b253db4e0db8ba0   
1     4fc10a6b85e5fa688b253db4e0db8ba0   
2     4fc10a6b85e5fa688b253db4e0db8ba0   
3     4fc10a6b85e5fa688b253

## Finishing Up

When you are finished, you should close the connection:

In [21]:
conn.close()